In [ ]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Conv2D,Dropout,Flatten,MaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger,ModelCheckpoint
from PIL import Image



### Build & compile model

In [ ]:
@tf.function
def num_char_acc(y_true,y_pred):
    y_true = tf.convert_to_tensor(y_true)
    _,num = tf.split(y_true, [40,4], 1)
    _,NUM = tf.split(y_pred, [40,4], 1)
    num_char_true = tf.math.argmax(num,axis=-1,output_type=tf.int32)
    num_char_pred = tf.math.argmax(NUM,axis=-1,output_type=tf.int32)
    n =tf.cast(num_char_true==num_char_pred,tf.float32)
    return tf.reduce_mean(n)

@tf.function
def custom_acc(y_true,y_pred):

    y_true = tf.convert_to_tensor(y_true)
    a,b,c,d ,num = tf.split(y_true, [10,10,10,10,4], 1)
    true = [a,b,c,d]
    A,B,C,D,_ = tf.split(y_pred, [10,10,10,10,4], 1)
    pred = [A,B,C,D]
    num_char_true = tf.math.argmax(num,axis=-1,output_type=tf.int32)+ 1
    i = 0 
    n =tf.constant(0,tf.int32)
    while (i<4) :
        char_pred = tf.math.argmax(pred[i],axis=-1,output_type=tf.int32)
        char_true = tf.math.argmax(true[i],axis=-1,output_type=tf.int32)
        n = n+ tf.reduce_sum(tf.cast((char_pred==char_true)&(tf.convert_to_tensor(i)<num_char_true),tf.int32))
        i=i+1
    accuracy = tf.divide(tf.cast(n,tf.float32) , tf.cast(tf.reduce_sum(num_char_true),tf.float32))
    return accuracy

@tf.function
def custom_loss(y_true,y_pred):
    alpha =5
    y_true = tf.convert_to_tensor(y_true)
    a,b,c,d ,num = tf.split(y_true, [10,10,10,10,4], 1)
    true = [a,b,c,d]
    A,B,C,D,NUM = tf.split(y_pred, [10,10,10,10,4], 1)
    pred = [A,B,C,D]
    num_char_true = tf.math.argmax(num,axis=-1,output_type=tf.int32)+ 1
    loss_1 = keras.losses.categorical_crossentropy(num ,NUM)
    i =0
    loss =tf.zeros_like(loss_1,dtype=tf.float32)
    #loss =tf.zeros([32,1],tf.float32)
    while (i<4) :
        char_pred = pred[i]
        char_true = true[i]
        loss = loss + tf.cast(tf.convert_to_tensor(i)<num_char_true,tf.float32)*keras.losses.categorical_crossentropy(char_true,char_pred)
        i=i+1
    return alpha * tf.reduce_sum(loss_1) + tf.reduce_sum(loss) 

def build_and_compile_model():
    
    rate =0.2
    rate2 = 0.5
    inputs = keras.Input(shape=(128, 128,1), name='img')
    #x = layers.Conv2D(16, 5, activation='relu',kernel_constraint=MaxNorm(3),padding='same')(inputs)
    x = Conv2D(16, 5, activation='relu',padding='same')(inputs)
    x = Dropout(rate)(x)
    x = Conv2D(16, 5, activation='relu',padding='same')(x)
    x = Dropout(rate)(x)
    x = MaxPooling2D(4)(x)
    x = Conv2D(32, 5, activation='relu',padding='same')(x)
    x = Dropout(rate)(x)
    x = Conv2D(32, 5, activation='relu',padding='same')(x)
    x = Dropout(rate)(x)
    x = MaxPooling2D(4)(x)
    x = Conv2D(64, 5, activation='relu',padding='same')(x)
    x = Dropout(rate)(x)
    x = Conv2D(64, 5, activation='relu',padding='same')(x)
    x = Dropout(rate)(x)
    x = MaxPooling2D(4)(x)
   
    feature = Flatten()(x)
    feature = Dropout(rate2)(feature)
    outputs = layers.concatenate([Dense(10, activation='softmax')(feature),Dense(10, activation='softmax')(feature),
                                  Dense(10, activation='softmax')(feature),Dense(10, activation='softmax')(feature),
                                  Dense(4, activation='softmax', name='num_digits')(feature)])
    
    model = keras.Model(inputs=inputs , outputs=outputs, name='mnist_model')
    model.compile(loss=custom_loss,optimizer=keras.optimizers.RMSprop(learning_rate=0.001)
                  ,metrics=[custom_acc,num_char_acc])
    return model

model = build_and_compile_model()
model.summary()

In [ ]:

def parse_func(s_example):
    features = {
                'label':tf.io.FixedLenFeature((44,),tf.float32),
                'image':tf.io.FixedLenFeature((128,128,1),tf.float32),
                }
    example = tf.io.parse_single_example(s_example, features=features)
    return example['image'],example['label']


### Data synthesization: <br/>variable-length handwritten digits randomly displayed (ex:varying  orientation  and translation)

In [ ]:
FILES = '/kaggle/input/variablelength-handwritten-digits/digits-*.tfrecords'
BATCH_SIZE=128
dataset = tf.data.Dataset.list_files(FILES).\
    interleave(tf.data.TFRecordDataset, cycle_length=4, block_length=1).map(parse_func)

train_dataset = dataset.skip(6000).repeat().shuffle(4000).batch(BATCH_SIZE,drop_remainder=True)
val_dataset = dataset.take(6000).batch(250,drop_remainder=True)

### Train the model

In [ ]:
#model = keras.models.load_model('/kaggle/input/digits-recog/model.h5',compile=False)
#model.compile(loss=custom_loss,optimizer=keras.optimizers.RMSprop(learning_rate=0.001),metrics=[custom_acc,num_char_acc])
#model.save_weights('/kaggle/working/chkpt')
#model.load_weights('/kaggle/input/recognition-rev/chkpt')
callbacks = [#ModelCheckpoint(filepath='/kaggle/working/chkpt',save_weights_only=True),
             #EarlyStopping(monitor='val_custom_acc', mode='max', patience=10),
             CSVLogger('training.log')]
model.fit(train_dataset,validation_data=val_dataset, epochs=50, steps_per_epoch=24000//BATCH_SIZE, callbacks=callbacks)
#model.fit(new_data,new_label,batch_size=128, epochs=70, validation_split=0.2, callbacks=callbacks)
model.save('/kaggle/working/model.h5')
model.save_weights('/kaggle/working/chkpt')

### Plot traing trace

In [ ]:
df =pd.read_csv('/kaggle/working/training.log')
df.set_index('epoch',inplace=True)
df[['custom_acc','val_custom_acc']].plot()
df[['loss','val_loss']].plot()